## Parse final GFF

In [18]:
! ls -al Slat_v4_repDNA_annot_Oct_2023.gff

-rw-rw-r-- 1 pavel pavel 595345692 říj 17 14:55 Slat_v4_repDNA_annot_Oct_2023.gff


In [19]:
fa = "Slat_v4_repDNA_annot_Oct_2023.gff"

In [20]:
! mkdir analysis_v3

mkdir: cannot create directory ‘analysis_v3’: File exists


In [21]:
# subsets
! grep "DNA\/[DH]" {fa} > analysis_v3/DNA_TEs_subset.gff
! grep "MITE\/" {fa} > analysis_v3/MITE_TEs_subset.gff

In [22]:
! grep "LTR\/" {fa} > analysis_v3/LTR_TEs_subset.gff

In [23]:
! grep "LINE" {fa} > analysis_v3/LINE_subset.gff

In [24]:
! grep "Satellites" {fa} > analysis_v3/satellites_subset.gff

In [25]:
! grep "rDNA" {fa} > analysis_v3/rdna_subset.gff

In [26]:
import os
destDir = os.getcwd() + '/analysis_v3'
os.chdir(destDir)

In [27]:
print(os.getcwd())

/mnt/hdd_6TB/40_Slat_assembly_v4/RepMask_newEDTA_merge_Oct_2023/analysis_v3


In [28]:
! ls *gff

DNA_TEs_subset.gff  LTR_TEs_subset.gff	 rdna_subset.gff
LINE_subset.gff     MITE_TEs_subset.gff  satellites_subset.gff


In [30]:
from Bio import SeqIO

genome_fa = "../../S.latifolia_v4.0.genome.fna"
chrLen = {}
noNs_chrLen = {}
for r in SeqIO.parse(genome_fa,"fasta"):
    chrLen[r.id] = len(r.seq)
    noNs_chrLen[r.id] = sum([r.seq.count(b) for b in 'ACGT'])
print(chrLen,'\n',noNs_chrLen)

{'chr1': 200709468, 'chr10': 165488795, 'chr11': 209616475, 'chr2': 199635652, 'chr3': 153359320, 'chr4': 112764170, 'chr5': 91147008, 'chr6': 141279031, 'chr7': 128518071, 'chr8': 113857337, 'chr9': 226323612, 'chrX': 346484273, 'chrY': 497814031} 
 {'chr1': 200602813, 'chr10': 165474827, 'chr11': 209211419, 'chr2': 199336714, 'chr3': 152962667, 'chr4': 112739747, 'chr5': 87672558, 'chr6': 138463458, 'chr7': 127163725, 'chr8': 110818139, 'chr9': 225456203, 'chrX': 323749825, 'chrY': 494938722}


In [31]:
seq_d = {ch:[chrLen[ch],noNs_chrLen[ch],0] for ch in chrLen}
seq_d

{'chr1': [200709468, 200602813, 0],
 'chr10': [165488795, 165474827, 0],
 'chr11': [209616475, 209211419, 0],
 'chr2': [199635652, 199336714, 0],
 'chr3': [153359320, 152962667, 0],
 'chr4': [112764170, 112739747, 0],
 'chr5': [91147008, 87672558, 0],
 'chr6': [141279031, 138463458, 0],
 'chr7': [128518071, 127163725, 0],
 'chr8': [113857337, 110818139, 0],
 'chr9': [226323612, 225456203, 0],
 'chrX': [346484273, 323749825, 0],
 'chrY': [497814031, 494938722, 0]}

## LTR superfamilies

In [32]:
! mkdir LTR_subsets

In [33]:
# within LTR subset
## 3 separate gffs: Gypsy, Copia, Unknown
! mkdir LTR_subsets
! grep "\/Gypsy" LTR_TEs_subset.gff > LTR_subsets/Gypsy_subset.gff
! grep "\/Copia" LTR_TEs_subset.gff > LTR_subsets/Copia_subset.gff
! grep "\/unknown" LTR_TEs_subset.gff > LTR_subsets/Unknown_subset.gff

mkdir: cannot create directory ‘LTR_subsets’: File exists


In [38]:
def get_repArea(merged_bed, ch):
    area = 0
    with open(merged_bed) as bed:
        for l in bed:
            if l.startswith(ch+"\t"):
                area += (int(l.split("\t")[2])-int(l.split("\t")[1]))+1
    return area

def get_chStat(ch):
    if ch == 'chrX':
        return 'X'
    elif ch == 'chrY':
        return 'Y'
    else:
        return 'A'

In [39]:
import glob

axy_dict = {}
for gff in glob.glob("LTR_subsets/*.gff"):
    merged_bed = gff.split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = gff.split("/")[1].split("_")[0]
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,2)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,2)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}")    

chr1	Copia	33311645	200709468	16.6	200602813	16.61
chr10	Copia	24257409	165488795	14.66	165474827	14.66
chr11	Copia	38451789	209616475	18.34	209211419	18.38
chr2	Copia	32501634	199635652	16.28	199336714	16.3
chr3	Copia	25817784	153359320	16.83	152962667	16.88
chr4	Copia	18199547	112764170	16.14	112739747	16.14
chr5	Copia	12255555	91147008	13.45	87672558	13.98
chr6	Copia	23583072	141279031	16.69	138463458	17.03
chr7	Copia	20549418	128518071	15.99	127163725	16.16
chr8	Copia	17837076	113857337	15.67	110818139	16.1
chr9	Copia	43385557	226323612	19.17	225456203	19.24
chrX	Copia	53965831	346484273	15.58	323749825	16.67
chrY	Copia	108079573	497814031	21.71	494938722	21.84
chr1	Gypsy	71223695	200709468	35.49	200602813	35.5
chr10	Gypsy	56816897	165488795	34.33	165474827	34.34
chr11	Gypsy	74273587	209616475	35.43	209211419	35.5
chr2	Gypsy	70529044	199635652	35.33	199336714	35.38
chr3	Gypsy	55166013	153359320	35.97	152962667	36.07
chr4	Gypsy	40130196	112764170	35.59	112739747	35.6
chr5	Gypsy	2915

In [40]:
# 2. LTR superfamilies piePlot
with open("Slat_LTR_superFams_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,2)
            noNs_perc = round((rep_area/noNs_chLen)*100,2)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## LTR families (lineages)

In [41]:
# LTR families:
ltr_fams = ["Class_I|LTR|Ty1/copia", "Class_I|LTR|Ty1/copia|Ale", "Class_I|LTR|Ty1/copia|Alesia", 
            "Class_I|LTR|Ty1/copia|Angela", "Class_I|LTR|Ty1/copia|Bianca", "Class_I|LTR|Ty1/copia|Ikeros", 
            "Class_I|LTR|Ty1/copia|Ivana", "Class_I|LTR|Ty1/copia|SIRE", "Class_I|LTR|Ty1/copia|TAR", 
            "Class_I|LTR|Ty1/copia|Tork", "Class_I|LTR|Ty3/gypsy", "Class_I|LTR|Ty3/gypsy|chromovirus", 
            "Class_I|LTR|Ty3/gypsy|chromovirus|CRM", "Class_I|LTR|Ty3/gypsy|chromovirus|Galadriel", 
            "Class_I|LTR|Ty3/gypsy|chromovirus|Reina", "Class_I|LTR|Ty3/gypsy|chromovirus|Tekay", 
            "Class_I|LTR|Ty3/gypsy|non-chromovirus|OTA|Athila", "Class_I|LTR|Ty3/gypsy|non-chromovirus|OTA|Tat", 
            "Class_I|LTR|Ty3/gypsy|non-chromovirus|OTA|Tat|Ogre", 
            "Class_I|LTR|Ty3/gypsy|non-chromovirus|OTA|Tat|Retand", "Class_I|LTR|Ty3/gypsy|non-chromovirus|Phygy"]

In [42]:
os.getcwd()

'/mnt/hdd_6TB/40_Slat_assembly_v4/RepMask_newEDTA_merge_Oct_2023/analysis_v3'

In [43]:
from collections import namedtuple

# define namedtuple for each track line
GffLine = namedtuple('GffLine', ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'])

def attributes2dict(attributes):
    """transform attributes to dictionary"""
    return {rec.split("=")[0]:rec.split("=")[1] for rec in attributes.split(";")}

In [44]:
! pwd

/mnt/hdd_6TB/40_Slat_assembly_v4/RepMask_newEDTA_merge_Oct_2023/analysis_v3


In [45]:
da_dict = {fam:{'chr1': [200709468, 200602813, 0],
 'chr10': [165488795, 165474827, 0],
 'chr11': [209616475, 209211419, 0],
 'chr2': [199635652, 199336714, 0],
 'chr3': [153359320, 152962667, 0],
 'chr4': [112764170, 112739747, 0],
 'chr5': [91147008, 87672558, 0],
 'chr6': [141279031, 138463458, 0],
 'chr7': [128518071, 127163725, 0],
 'chr8': [113857337, 110818139, 0],
 'chr9': [226323612, 225456203, 0],
 'chrX': [346484273, 323749825, 0],
 'chrY': [497814031, 494938722, 0]} for fam in ltr_fams}
da_dict

{'Class_I|LTR|Ty1/copia': {'chr1': [200709468, 200602813, 0],
  'chr10': [165488795, 165474827, 0],
  'chr11': [209616475, 209211419, 0],
  'chr2': [199635652, 199336714, 0],
  'chr3': [153359320, 152962667, 0],
  'chr4': [112764170, 112739747, 0],
  'chr5': [91147008, 87672558, 0],
  'chr6': [141279031, 138463458, 0],
  'chr7': [128518071, 127163725, 0],
  'chr8': [113857337, 110818139, 0],
  'chr9': [226323612, 225456203, 0],
  'chrX': [346484273, 323749825, 0],
  'chrY': [497814031, 494938722, 0]},
 'Class_I|LTR|Ty1/copia|Ale': {'chr1': [200709468, 200602813, 0],
  'chr10': [165488795, 165474827, 0],
  'chr11': [209616475, 209211419, 0],
  'chr2': [199635652, 199336714, 0],
  'chr3': [153359320, 152962667, 0],
  'chr4': [112764170, 112739747, 0],
  'chr5': [91147008, 87672558, 0],
  'chr6': [141279031, 138463458, 0],
  'chr7': [128518071, 127163725, 0],
  'chr8': [113857337, 110818139, 0],
  'chr9': [226323612, 225456203, 0],
  'chrX': [346484273, 323749825, 0],
  'chrY': [497814031

In [46]:
import glob

for gff in glob.glob("LTR_subsets/*_subset.gff"):
    ann = gff.split("/")[1].split("_")[0]
    with open(gff) as gf:
        for l in gf:
            if 'Dante_annotation' in l:
                ll = l.rstrip().split("\t")
                ll[-1] = attributes2dict(ll[-1])
                gfl = GffLine(*ll)
                if gfl.attributes['Dante_annotation'] in [k for k in ltr_fams] and 'retrotransposon' in gfl.type:
                    rep_area = (int(gfl.end)-int(gfl.start))+1
                    da_dict[gfl.attributes['Dante_annotation']][gfl.seqid][2] += rep_area

In [47]:
# Filtered for the most abundant families: 'Ale','Angela','Athila','CRM','Ogre','Retand','SIRE','Tat' and 'Tekay' 

for fam in da_dict:
    for ch in da_dict[fam]:
        fm = fam.split("|")[2].split("/")[1] + "_" + fam.split("|")[-1]
        if fm.split("_")[1] in ['Ale','Angela','Athila','CRM','Ogre','Retand','SIRE','Tat','Tekay']:
            proc1 = (da_dict[fam][ch][2]/da_dict[fam][ch][0])*100
            proc2 = (da_dict[fam][ch][2]/da_dict[fam][ch][1])*100
            print(f"{ch}\t{fm}\t{da_dict[fam][ch][2]}\t{da_dict[fam][ch][0]}\t{proc1}\t{da_dict[fam][ch][1]}\t{proc2}")

chr1	copia_Ale	0	200709468	0.0	200602813	0.0
chr10	copia_Ale	0	165488795	0.0	165474827	0.0
chr11	copia_Ale	0	209616475	0.0	209211419	0.0
chr2	copia_Ale	0	199635652	0.0	199336714	0.0
chr3	copia_Ale	0	153359320	0.0	152962667	0.0
chr4	copia_Ale	0	112764170	0.0	112739747	0.0
chr5	copia_Ale	0	91147008	0.0	87672558	0.0
chr6	copia_Ale	0	141279031	0.0	138463458	0.0
chr7	copia_Ale	0	128518071	0.0	127163725	0.0
chr8	copia_Ale	0	113857337	0.0	110818139	0.0
chr9	copia_Ale	0	226323612	0.0	225456203	0.0
chrX	copia_Ale	0	346484273	0.0	323749825	0.0
chrY	copia_Ale	0	497814031	0.0	494938722	0.0
chr1	copia_Angela	0	200709468	0.0	200602813	0.0
chr10	copia_Angela	0	165488795	0.0	165474827	0.0
chr11	copia_Angela	0	209616475	0.0	209211419	0.0
chr2	copia_Angela	0	199635652	0.0	199336714	0.0
chr3	copia_Angela	0	153359320	0.0	152962667	0.0
chr4	copia_Angela	0	112764170	0.0	112739747	0.0
chr5	copia_Angela	0	91147008	0.0	87672558	0.0
chr6	copia_Angela	0	141279031	0.0	138463458	0.0
chr7	copia_Angela	0	128518071	0

In [51]:
seqLen_d = {'chr1': [200709468, 200602813, 0],
 'chr10': [165488795, 165474827, 0],
 'chr11': [209616475, 209211419, 0],
 'chr2': [199635652, 199336714, 0],
 'chr3': [153359320, 152962667, 0],
 'chr4': [112764170, 112739747, 0],
 'chr5': [91147008, 87672558, 0],
 'chr6': [141279031, 138463458, 0],
 'chr7': [128518071, 127163725, 0],
 'chr8': [113857337, 110818139, 0],
 'chr9': [226323612, 225456203, 0],
 'chrX': [346484273, 323749825, 0],
 'chrY': [497814031, 494938722, 0]}

def get_chStat(ch):
    if ch == 'chrX':
        return 'X'
    elif ch == 'chrY':
        return 'Y'
    else:
        return 'A'
    
out_d = {'A':[0,0],'X':[0,0],'Y':[0,0]}
for ch in seqLen_d:
    chs = get_chStat(ch)
    out_d[chs][0] += seqLen_d[ch][0]
    out_d[chs][1] += seqLen_d[ch][1]
print(out_d)

{'A': [1742698939, 1729902270], 'X': [346484273, 323749825], 'Y': [497814031, 494938722]}


In [53]:
import glob
ltr_fam_dict = {k:{'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]} for k in ltr_fams}
for gff in glob.glob("LTR_subsets/*_subset.gff"):
    
    ann = gff.split("/")[1].split("_")[0]
    print(gff,ann)
    with open(gff) as gf:
        for l in gf:
            if 'dante_annotation' in l:
                ll = l.rstrip().split("\t")
                ll[-1] = attributes2dict(ll[-1])
                gfl = GffLine(*ll)
                if gfl.attributes['dante_annotation'] in [k for k in ltr_fam_dict] and 'retrotransposon' in gfl.type:
                    chStat = get_chStat(gfl.seqid)
                    rep_area = (int(gfl.end)-int(gfl.start))+1
                    #print(chStat, ltr_fam_dict[gfl.attributes['Dante_annotation']])
                    ltr_fam_dict[gfl.attributes['dante_annotation']][chStat][0] += rep_area
                    
for ann in ltr_fam_dict:
    ltr_fam_dict[ann]['A'][1]=1742698939
    ltr_fam_dict[ann]['A'][2]=1729902270
    ltr_fam_dict[ann]['X'][1]=346484273
    ltr_fam_dict[ann]['X'][2]=323749825
    ltr_fam_dict[ann]['Y'][1]=497814031
    ltr_fam_dict[ann]['Y'][2]=494938722
print(ltr_fam_dict)


LTR_subsets/Copia_subset.gff Copia
LTR_subsets/Gypsy_subset.gff Gypsy
LTR_subsets/Unknown_subset.gff Unknown
{'Class_I|LTR|Ty1/copia': {'A': [3751925, 1742698939, 1729902270], 'X': [482392, 346484273, 323749825], 'Y': [1188790, 497814031, 494938722]}, 'Class_I|LTR|Ty1/copia|Ale': {'A': [19233809, 1742698939, 1729902270], 'X': [2838425, 346484273, 323749825], 'Y': [19650747, 497814031, 494938722]}, 'Class_I|LTR|Ty1/copia|Alesia': {'A': [42679, 1742698939, 1729902270], 'X': [3516, 346484273, 323749825], 'Y': [34447, 497814031, 494938722]}, 'Class_I|LTR|Ty1/copia|Angela': {'A': [179141682, 1742698939, 1729902270], 'X': [31024225, 346484273, 323749825], 'Y': [39432455, 497814031, 494938722]}, 'Class_I|LTR|Ty1/copia|Bianca': {'A': [555442, 1742698939, 1729902270], 'X': [228182, 346484273, 323749825], 'Y': [16921, 497814031, 494938722]}, 'Class_I|LTR|Ty1/copia|Ikeros': {'A': [3759171, 1742698939, 1729902270], 'X': [761386, 346484273, 323749825], 'Y': [1324439, 497814031, 494938722]}, 'Class_

In [54]:
# 3. LTR families plots
with open("Slat_LTR_families_proportion_v1.tab","w") as out:
    for ann in ltr_fam_dict:
        for chStat in ltr_fam_dict[ann]:
            rep_area = ltr_fam_dict[ann][chStat][0]
            chLen = ltr_fam_dict[ann][chStat][1]
            noNs_chLen = ltr_fam_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,2)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            annL = ann.split("/")[1].split("|")
            if len(annL) > 1:
                annF = annL[0]+"_"+annL[-1]
                out.write(f"{annF}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## DNA TEs

In [56]:
! rm -rf DNA_supfams/
! mkdir DNA_supfams 
! grep 'DNA/DTA' DNA_TEs_subset.gff > DNA_supfams/DTA_subset.gff
! grep 'DNA/DTC' DNA_TEs_subset.gff > DNA_supfams/DTC_subset.gff
! grep 'DNA/DTH' DNA_TEs_subset.gff > DNA_supfams/DTH_subset.gff
! grep 'DNA/DTM' DNA_TEs_subset.gff > DNA_supfams/DTM_subset.gff
! grep 'DNA/DTT' DNA_TEs_subset.gff > DNA_supfams/DTT_subset.gff
! grep 'DNA/Helitron' ../Slat_v4_repDNA_annot_Oct_2023.gff > DNA_supfams/Helitron_subset.gff

In [57]:
# DNA autonomous TEs
annot_dict = {"DNA/DTA":"hAT", "DNA/DTC":"CACTA", "DNA/DTH":"PIF/Harbinger", 
              "DNA/DTM":"Mutator", "DNA/DTT":"Tcl/Mariner",
              "DNA/Helitron":"Helitron", "LTR/Copia":"Ty1/Copia", "LTR/Gypsy":"Ty3/Gypsy", 
              "LTR/unknown":"Unknown",
              "MITE/DTA":"hAT", "MITE/DTC":"CACTA", "MITE/DTH":"PIF/Harbinger",
              "MITE/DTM":"Mutator", "MITE/DTT":"Tcl/Mariner"}
axy_dict = {}
for gff in glob.glob("DNA_supfams/*.gff"):
    merged_bed = gff.split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = annot_dict["DNA/"+gff.split("/")[1].split("_")[0]]
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,2)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,2)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}")    

chr1	CACTA	3263529	200709468	1.63	200602813	1.63
chr10	CACTA	2162567	165488795	1.31	165474827	1.31
chr11	CACTA	3248156	209616475	1.55	209211419	1.55
chr2	CACTA	2305630	199635652	1.15	199336714	1.16
chr3	CACTA	1918569	153359320	1.25	152962667	1.25
chr4	CACTA	883350	112764170	0.78	112739747	0.78
chr5	CACTA	444448	91147008	0.49	87672558	0.51
chr6	CACTA	1904582	141279031	1.35	138463458	1.38
chr7	CACTA	1385987	128518071	1.08	127163725	1.09
chr8	CACTA	1481675	113857337	1.3	110818139	1.34
chr9	CACTA	3170831	226323612	1.4	225456203	1.41
chrX	CACTA	5354296	346484273	1.55	323749825	1.65
chrY	CACTA	14269790	497814031	2.87	494938722	2.88
chr1	Mutator	2989985	200709468	1.49	200602813	1.49
chr10	Mutator	4102021	165488795	2.48	165474827	2.48
chr11	Mutator	6796060	209616475	3.24	209211419	3.25
chr2	Mutator	2972496	199635652	1.49	199336714	1.49
chr3	Mutator	2899172	153359320	1.89	152962667	1.9
chr4	Mutator	1180460	112764170	1.05	112739747	1.05
chr5	Mutator	507552	91147008	0.56	87672558	0.58
chr6	Mutato

In [58]:
# DNA TEs tab:
with open("Slat_DNA_autonomous_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,2)
            noNs_perc = round((rep_area/noNs_chLen)*100,2)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## MITEs

In [59]:
! rm -rf MITE_supfams/
! mkdir MITE_supfams 
! grep 'MITE/DTA' MITE_TEs_subset.gff > MITE_supfams/DTA_subset.gff
! grep 'MITE/DTC' MITE_TEs_subset.gff > MITE_supfams/DTC_subset.gff
! grep 'MITE/DTH' MITE_TEs_subset.gff > MITE_supfams/DTH_subset.gff
! grep 'MITE/DTM' MITE_TEs_subset.gff > MITE_supfams/DTM_subset.gff
! grep 'MITE/DTT' MITE_TEs_subset.gff > MITE_supfams/DTT_subset.gff

In [60]:
# DNA autonomous TEs
annot_dict = {"DNA/DTA":"hAT", "DNA/DTC":"CACTA", "DNA/DTH":"PIF/Harbinger", 
              "DNA/DTM":"Mutator", "DNA/DTT":"Tcl/Mariner",
              "DNA/Helitron":"Helitron", "LTR/Copia":"Ty1/Copia", "LTR/Gypsy":"Ty3/Gypsy", 
              "LTR/unknown":"Unknown",
              "MITE/DTA":"hAT", "MITE/DTC":"CACTA", "MITE/DTH":"PIF/Harbinger",
              "MITE/DTM":"Mutator", "MITE/DTT":"Tcl/Mariner"}
axy_dict = {}
for gff in glob.glob("MITE_supfams/*.gff"):
    merged_bed = gff.split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = annot_dict["MITE/"+gff.split("/")[1].split("_")[0]]
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,4)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,4)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}") 

chr1	CACTA	7489	200709468	0.0037	200602813	0.0037
chr10	CACTA	61561	165488795	0.0372	165474827	0.0372
chr11	CACTA	1121	209616475	0.0005	209211419	0.0005
chr2	CACTA	3274	199635652	0.0016	199336714	0.0016
chr3	CACTA	2709	153359320	0.0018	152962667	0.0018
chr4	CACTA	8952	112764170	0.0079	112739747	0.0079
chr5	CACTA	10904	91147008	0.012	87672558	0.0124
chr6	CACTA	8277	141279031	0.0059	138463458	0.006
chr7	CACTA	16361	128518071	0.0127	127163725	0.0129
chr8	CACTA	2026	113857337	0.0018	110818139	0.0018
chr9	CACTA	25586	226323612	0.0113	225456203	0.0113
chrX	CACTA	25475	346484273	0.0074	323749825	0.0079
chrY	CACTA	6623	497814031	0.0013	494938722	0.0013
chr1	Mutator	268761	200709468	0.1339	200602813	0.134
chr10	Mutator	6042901	165488795	3.6515	165474827	3.6519
chr11	Mutator	1174205	209616475	0.5602	209211419	0.5613
chr2	Mutator	2649403	199635652	1.3271	199336714	1.3291
chr3	Mutator	280505	153359320	0.1829	152962667	0.1834
chr4	Mutator	310526	112764170	0.2754	112739747	0.2754
chr5	Mutator	175910

In [61]:
# MITE TEs tab:
with open("Slat_DNA_MITE_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,4)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## Satellites

In [170]:
# satellites
satHeads = ["CL12_25SrDNA+TR1+X431_8470bp#rDNATR1X431", "CL12_X43.1_312bp#X431CL12", "CL12_TR1_74bp#TR1CL12", 
    "CL16_STAR_43bp#STARCL16", "CL68_TRAYC_165bp#TRAYCCL68", 
    "CL87_15Ssp_159bp#15SCL87", "CL99_5SrDNA+spacer_364bp#5SCL99", "SIlL1#CEN", "SilL2#CEN"]

subSat = [sat.split("#")[0] for sat in satHeads]
subSat

['CL12_25SrDNA+TR1+X431_8470bp',
 'CL12_X43.1_312bp',
 'CL12_TR1_74bp',
 'CL16_STAR_43bp',
 'CL68_TRAYC_165bp',
 'CL87_15Ssp_159bp',
 'CL99_5SrDNA+spacer_364bp',
 'SIlL1',
 'SilL2']

In [62]:
! mkdir SAT_subsets

In [63]:
sat_dict = {"Satellites/STAR-C":"STAR-C","Satellites/STAR-Y":"STAR-Y","Satellites/TR1":"TR1","Satellites/TRAYC":"TRAYC","Satellites/X43.1":"X43.1"}

for sat in sat_dict:
    out_name = 'SAT_subsets/'+sat_dict[sat]+'_subset.gff'
    with open(out_name,'w') as out:
        with open("satellites_subset.gff") as gff:
            for l in gff:
                if sat in l:
                    out.write(l)

In [64]:
axy_dict = {}
for gff in glob.glob("SAT_subsets/*.gff"):
    merged_bed = gff.split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = gff.split("/")[1].split("_")[0]
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,4)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,4)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}") 

chr1	STAR-C	1721282	200709468	0.8576	200602813	0.8581
chr10	STAR-C	728037	165488795	0.4399	165474827	0.44
chr11	STAR-C	1478224	209616475	0.7052	209211419	0.7066
chr2	STAR-C	1408831	199635652	0.7057	199336714	0.7068
chr3	STAR-C	1530167	153359320	0.9978	152962667	1.0004
chr4	STAR-C	337058	112764170	0.2989	112739747	0.299
chr5	STAR-C	50005	91147008	0.0549	87672558	0.057
chr6	STAR-C	1188794	141279031	0.8415	138463458	0.8586
chr7	STAR-C	99874	128518071	0.0777	127163725	0.0785
chr8	STAR-C	57701	113857337	0.0507	110818139	0.0521
chr9	STAR-C	4076308	226323612	1.8011	225456203	1.808
chrX	STAR-C	477991	346484273	0.138	323749825	0.1476
chrY	STAR-C	701719	497814031	0.141	494938722	0.1418
chr1	STAR-Y	46	200709468	0.0	200602813	0.0
chr10	STAR-Y	933	165488795	0.0006	165474827	0.0006
chr11	STAR-Y	552	209616475	0.0003	209211419	0.0003
chr2	STAR-Y	2408	199635652	0.0012	199336714	0.0012
chr3	STAR-Y	592	153359320	0.0004	152962667	0.0004
chr4	STAR-Y	52	112764170	0.0	112739747	0.0
chr5	STAR-Y	0	91147008	0.0

In [65]:
# Satellites tab:
with open("Slat_Satellites_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,4)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## LINEs

In [67]:
! mkdir LINE_subsets

In [68]:
# LINE
lineHeads=["CL55_LINE_3458bp#LINECL55", "CL84_LINE_3865bp#LINECL84", "CL119_LINE_2075bp#LINECL119"]
lineDict = {v.split("#")[0]:v.split("#")[1] for v in lineHeads}
for line in lineDict:
    out_name = 'LINE_subsets/'+lineDict[line]+'_subset.gff'
    with open(out_name,'w') as out:
        with open("LINE_subset.gff") as gff:
            for l in gff:
                if 'Motif:'+line in l:
                    out.write(l)
                elif lineDict[line] in l:
                    out.write(l)

In [69]:
axy_dict = {}
for gff in glob.glob("LINE_subsets/*.gff"):
    merged_bed = gff.split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = gff.split("/")[1].split("_")[0]
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,4)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,4)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}") 

chr1	LINECL84	282872	200709468	0.1409	200602813	0.141
chr10	LINECL84	227574	165488795	0.1375	165474827	0.1375
chr11	LINECL84	290044	209616475	0.1384	209211419	0.1386
chr2	LINECL84	299843	199635652	0.1502	199336714	0.1504
chr3	LINECL84	202756	153359320	0.1322	152962667	0.1326
chr4	LINECL84	147524	112764170	0.1308	112739747	0.1309
chr5	LINECL84	137075	91147008	0.1504	87672558	0.1563
chr6	LINECL84	201537	141279031	0.1427	138463458	0.1456
chr7	LINECL84	203710	128518071	0.1585	127163725	0.1602
chr8	LINECL84	156389	113857337	0.1374	110818139	0.1411
chr9	LINECL84	240990	226323612	0.1065	225456203	0.1069
chrX	LINECL84	298149	346484273	0.086	323749825	0.0921
chrY	LINECL84	672896	497814031	0.1352	494938722	0.136
chr1	LINECL55	693887	200709468	0.3457	200602813	0.3459
chr10	LINECL55	464221	165488795	0.2805	165474827	0.2805
chr11	LINECL55	988950	209616475	0.4718	209211419	0.4727
chr2	LINECL55	602491	199635652	0.3018	199336714	0.3022
chr3	LINECL55	433115	153359320	0.2824	152962667	0.2832
chr4	LINECL

In [70]:
# LINEs tab:
with open("Slat_LINE_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,4)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## rDNAs

In [72]:
! head rdna_subset.gff

chr1	RepeatMasker	similarity	81858311	81858625	 2.9	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|314
chr1	RepeatMasker	similarity	82167797	82168158	 5.5	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|351
chr1	RepeatMasker	similarity	82168159	82168514	 6.5	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|364
chr1	RepeatMasker	similarity	82168515	82168872	 4.2	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|364
chr1	RepeatMasker	similarity	82168873	82169236	 7.4	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|364
chr1	RepeatMasker	similarity	82169237	82169584	 7.2	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|364
chr1	RepeatMasker	similarity	82169585	82169946	 6.0	-	.	Classification=rDNA/5S;RM_attribute=Target|"Motif:CL99_5SrDNA+spacer_364bp"|1|364
chr1	RepeatMasker	similarity	82169

In [73]:
! mkdir rDNA_subsets

In [75]:
for rdna in ['rDNA/5S','rDNA/45S']:
    out_name = 'rDNA_subsets/'+rdna.split("/")[1]+'_subset.gff'
    with open(out_name,'w') as out:
        with open("rdna_subset.gff") as gff:
            for l in gff:
                if 'Classification='+rdna+";" in l:
                    out.write(l)
                elif lineDict[line] in l:
                    out.write(l)

In [76]:
axy_dict = {}
for gff in glob.glob("rDNA_subsets/*.gff"):
    merged_bed = gff.split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = gff.split("/")[1].split("_")[0]
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,4)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,4)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}") 

chr1	45S	89082	200709468	0.0444	200602813	0.0444
chr10	45S	0	165488795	0.0	165474827	0.0
chr11	45S	1282	209616475	0.0006	209211419	0.0006
chr2	45S	89458	199635652	0.0448	199336714	0.0449
chr3	45S	0	153359320	0.0	152962667	0.0
chr4	45S	0	112764170	0.0	112739747	0.0
chr5	45S	0	91147008	0.0	87672558	0.0
chr6	45S	37032	141279031	0.0262	138463458	0.0267
chr7	45S	0	128518071	0.0	127163725	0.0
chr8	45S	0	113857337	0.0	110818139	0.0
chr9	45S	147787	226323612	0.0653	225456203	0.0656
chrX	45S	16283	346484273	0.0047	323749825	0.005
chrY	45S	78416	497814031	0.0158	494938722	0.0158
chr1	5S	107100	200709468	0.0534	200602813	0.0534
chr10	5S	0	165488795	0.0	165474827	0.0
chr11	5S	362	209616475	0.0002	209211419	0.0002
chr2	5S	87049	199635652	0.0436	199336714	0.0437
chr3	5S	0	153359320	0.0	152962667	0.0
chr4	5S	2573	112764170	0.0023	112739747	0.0023
chr5	5S	0	91147008	0.0	87672558	0.0
chr6	5S	0	141279031	0.0	138463458	0.0
chr7	5S	0	128518071	0.0	127163725	0.0
chr8	5S	0	113857337	0.0	110818139	0.0
chr9	5

In [77]:
# rDNA tab:
with open("Slat_rDNA_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,4)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

## cpDNA

In [81]:
! head ../../nupts_numts_Oct2023/Slat_v4_cpDNA_RM.gff

chr1	RepeatMasker	similarity	54563	54668	 3.9	-	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|89062|89163
chr1	RepeatMasker	similarity	1107201	1107316	12.0	-	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|115349|115469
chr1	RepeatMasker	similarity	1141963	1142076	10.2	-	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|115349|115469
chr1	RepeatMasker	similarity	1289786	1289895	14.7	-	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|70070|70190
chr1	RepeatMasker	similarity	2195479	2195662	14.6	+	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|64399|64591
chr1	RepeatMasker	similarity	2885726	2885915	10.7	+	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|121043|121249
chr1	RepeatMasker	similarity	3304382	3304482	 3.0	-	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|52243|52343
chr1	RepeatMasker	similarity	3913280	3913501	 6.7	-	.	Classification=cpDNA;RM_attribute=Target|"Motif:cpDNA"|122576|122797
chr1	RepeatMasker	similarity

In [82]:
! mkdir cpDNA_subsets

In [83]:
axy_dict = {}
for gff in glob.glob("../../nupts_numts_Oct2023/Slat_v4_cpDNA_RM.gff"):
    merged_bed = gff.split("/")[-1].split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = "cpDNA"
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,4)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,4)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}") 

chr1	cpDNA	64092	200709468	0.0319	200602813	0.0319
chr10	cpDNA	46932	165488795	0.0284	165474827	0.0284
chr11	cpDNA	78633	209616475	0.0375	209211419	0.0376
chr2	cpDNA	83167	199635652	0.0417	199336714	0.0417
chr3	cpDNA	34885	153359320	0.0227	152962667	0.0228
chr4	cpDNA	71193	112764170	0.0631	112739747	0.0631
chr5	cpDNA	68198	91147008	0.0748	87672558	0.0778
chr6	cpDNA	45038	141279031	0.0319	138463458	0.0325
chr7	cpDNA	26988	128518071	0.021	127163725	0.0212
chr8	cpDNA	52147	113857337	0.0458	110818139	0.0471
chr9	cpDNA	231452	226323612	0.1023	225456203	0.1027
chrX	cpDNA	179287	346484273	0.0517	323749825	0.0554
chrY	cpDNA	637943	497814031	0.1281	494938722	0.1289


In [84]:
# cpDNA tab:
with open("Slat_cpDNA_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,4)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")

In [85]:
axy_dict = {}
for gff in glob.glob("../../nupts_numts_Oct2023/Slat_v4_mtDNA_RM.gff"):
    merged_bed = gff.split("/")[-1].split('.')[0]+"_merged.bed"
    cmd = f"bedtools merge -i {gff} > {merged_bed}"
    returned_value = os.system(cmd)  # returns the exit code in unix
    #print(f'CMD:{cmd}\nreturned value:{returned_value}')
    ann = "mtDNA"
    axy_dict[ann] = {'A':[0,0,0],'X':[0,0,0],'Y':[0,0,0]}
    for ch in chrLen:
        rep_area = get_repArea(merged_bed, ch)
        chStat = get_chStat(ch)
        axy_dict[ann][chStat][0] += rep_area
        axy_dict[ann][chStat][1] += chrLen[ch]
        axy_dict[ann][chStat][2] += noNs_chrLen[ch]
        #print(ann,chStat,ch,axy_dict[ann][chStat])
        #axy_dict[ann][chStat].append([rep_area,chrLen[ch],noNs_chrLen[ch]])
        perc = round((rep_area/chrLen[ch])*100,4)
        nns_perc = round((rep_area/noNs_chrLen[ch])*100,4)
        print(f"{ch}\t{ann}\t{rep_area}\t{chrLen[ch]}\t{perc}\t{noNs_chrLen[ch]}\t{nns_perc}") 

chr1	mtDNA	942167	200709468	0.4694	200602813	0.4697
chr10	mtDNA	762599	165488795	0.4608	165474827	0.4609
chr11	mtDNA	886111	209616475	0.4227	209211419	0.4235
chr2	mtDNA	1009067	199635652	0.5055	199336714	0.5062
chr3	mtDNA	764497	153359320	0.4985	152962667	0.4998
chr4	mtDNA	520085	112764170	0.4612	112739747	0.4613
chr5	mtDNA	500221	91147008	0.5488	87672558	0.5706
chr6	mtDNA	685105	141279031	0.4849	138463458	0.4948
chr7	mtDNA	516892	128518071	0.4022	127163725	0.4065
chr8	mtDNA	483587	113857337	0.4247	110818139	0.4364
chr9	mtDNA	989263	226323612	0.4371	225456203	0.4388
chrX	mtDNA	1239453	346484273	0.3577	323749825	0.3828
chrY	mtDNA	2312212	497814031	0.4645	494938722	0.4672


In [86]:
# mtDNA tab:
with open("Slat_mtDNA_proportion.tab","w") as out:
    for ann in axy_dict:
        for chStat in axy_dict[ann]:
            rep_area = axy_dict[ann][chStat][0]
            chLen = axy_dict[ann][chStat][1]
            noNs_chLen = axy_dict[ann][chStat][2]
            perc = round((rep_area/chLen)*100,4)
            noNs_perc = round((rep_area/noNs_chLen)*100,4)
            out.write(f"{ann}\t{chStat}\t{rep_area}\t{chLen}\t{noNs_chLen}\t{perc}\t{noNs_perc}\n")